In [0]:
sc

<SparkContext master=local[*, 4] appName=Databricks Shell>

In [0]:
spark

In [0]:
df = spark.read.csv("/Volumes/azuredatabricks212/default/datapractice/sales_data_sample.csv",inferSchema=True,header = True)
df.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003


### DATA CLEANING

In [0]:
from pyspark.sql.functions import *
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES|ORDERDATE|STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME|PHONE|ADDRESSLINE1|ADDRESSLINE2|CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|          0|              0|        0|              0|    0|        0|     0|     0|       0|      0|          0|   0|          0|           0|    0|           0|        2521|   0| 1486|        76|      0|        0|       

In [0]:
new_df = df.dropna()

In [0]:
new_df.select([count(when(isnull(c), c)).alias(c) for c in new_df.columns]).show()

+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES|ORDERDATE|STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME|PHONE|ADDRESSLINE1|ADDRESSLINE2|CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|          0|              0|        0|              0|    0|        0|     0|     0|       0|      0|          0|   0|          0|           0|    0|           0|           0|   0|    0|         0|      0|        0|       


### DATA PREPROCESSING

In [0]:
new_df.select(['PHONE']).show()

+---------------+
|          PHONE|
+---------------+
|   03 9520 4555|
|     2125551500|
|+61 2 9495 8555|
|     2125558493|
|+61 2 9495 8555|
|   03 9520 4555|
|   02 9936 8555|
|   03 9520 4555|
|   03 9520 4555|
|+61 2 9495 8555|
|   03 9520 4555|
|   03 9520 4555|
|     2125551500|
|     2125558493|
|+61 2 9495 8555|
|   03 9520 4555|
|   02 9936 8555|
|   02 9936 8555|
|   02 9936 8555|
|     2125557413|
+---------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import regexp_replace

new_df = new_df.withColumn('PHONE', regexp_replace('PHONE', '[^0-9]', ''))
new_df.select(['PHONE']).show()

+-----------+
|      PHONE|
+-----------+
| 0395204555|
| 2125551500|
|61294958555|
| 2125558493|
|61294958555|
| 0395204555|
| 0299368555|
| 0395204555|
| 0395204555|
|61294958555|
| 0395204555|
| 0395204555|
| 2125551500|
| 2125558493|
|61294958555|
| 0395204555|
| 0299368555|
| 0299368555|
| 0299368555|
| 2125557413|
+-----------+
only showing top 20 rows




### DATA OPTIMIZATION

In [0]:
from pyspark import StorageLevel
new_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[ORDERNUMBER: int, QUANTITYORDERED: int, PRICEEACH: double, ORDERLINENUMBER: int, SALES: double, ORDERDATE: string, STATUS: string, QTR_ID: int, MONTH_ID: int, YEAR_ID: int, PRODUCTLINE: string, MSRP: int, PRODUCTCODE: string, CUSTOMERNAME: string, PHONE: string, ADDRESSLINE1: string, ADDRESSLINE2: string, CITY: string, STATE: string, POSTALCODE: string, COUNTRY: string, TERRITORY: string, CONTACTLASTNAME: string, CONTACTFIRSTNAME: string, DEALSIZE: string]

In [0]:
new_df = new_df.repartition(200)
new_df.cache()
display(new_df)

ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
10420,36,63.57,4,2288.52,5/29/2005 0:00,In Process,2,5,2005,Vintage Cars,60,S18_2248,Souveniers And Things Co.,61294958555,"Monitor Money Building, 815 Pacific Hwy",Level 6,Chatswood,NSW,2067,Australia,APAC,Huxley,Adrian,Small
10204,45,49.81,12,2241.45,12/2/2003 0:00,Shipped,4,12,2003,Classic Cars,50,S24_1628,Muscle Machine Inc,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,NA,Young,Jeff,Small
10370,29,57.53,6,1668.37,1/20/2005 0:00,Shipped,1,1,2005,Trucks and Buses,121,S18_4600,"Anna's Decorations, Ltd",0299368555,201 Miller Street,Level 15,North Sydney,NSW,2060,Australia,APAC,O'Hara,Anna,Small
10370,27,100.0,1,3911.49,1/20/2005 0:00,Shipped,1,1,2005,Trucks and Buses,116,S18_1097,"Anna's Decorations, Ltd",0299368555,201 Miller Street,Level 15,North Sydney,NSW,2060,Australia,APAC,O'Hara,Anna,Medium
10127,39,38.19,12,1489.41,6/3/2003 0:00,Shipped,2,6,2003,Classic Cars,35,S24_2840,Muscle Machine Inc,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,NA,Young,Jeff,Small
10391,44,38.5,5,1694.0,3/9/2005 0:00,Shipped,1,3,2005,Trucks and Buses,60,S18_2432,"Anna's Decorations, Ltd",0299368555,201 Miller Street,Level 15,North Sydney,NSW,2060,Australia,APAC,O'Hara,Anna,Small
10223,47,100.0,9,4724.91,2/20/2004 0:00,Shipped,1,2,2004,Planes,84,S18_2581,"Australian Collectors, Co.",0395204555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,APAC,Ferguson,Peter,Medium
10223,34,100.0,11,3608.76,2/20/2004 0:00,Shipped,1,2,2004,Planes,109,S24_1785,"Australian Collectors, Co.",0395204555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,APAC,Ferguson,Peter,Medium
10324,27,100.0,12,3155.49,11/5/2004 0:00,Shipped,4,11,2004,Classic Cars,169,S18_3232,Vitachrome Inc.,2125551500,2678 Kingston Rd.,Suite 101,NYC,NY,10022,USA,NA,Frick,Michael,Medium
10204,39,33.23,2,1295.97,12/2/2003 0:00,Shipped,4,12,2003,Vintage Cars,41,S24_3969,Muscle Machine Inc,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,NA,Young,Jeff,Small


In [0]:
new_df.createOrReplaceTempView("SalesData")

In [0]:
from pyspark.sql.functions import *

spark.sql("SELECT QTR_ID,SUM(QUANTITYORDERED) AS sum_new_column FROM SalesData GROUP BY QTR_ID").show()

+------+--------------+
|QTR_ID|sum_new_column|
+------+--------------+
|     2|          2158|
|     4|          4015|
|     1|          1172|
|     3|          1314|
+------+--------------+




### MERGING AND JOINING OF DATASETS

In [0]:
df2 = spark.read.csv("/Volumes/azuredatabricks212/default/datapractice/HR Employee.xlsx - HR data.csv",inferSchema = True,header = True)

joined_DF = new_df.join(df2,how='inner')
joined_DF.show()

+-----------+---------------+---------+---------------+-------+--------------+----------+------+--------+-------+------------+----+-----------+--------------------+-----------+--------------------+------------+---------+-----+----------+---------+---------+---------------+----------------+--------+---------+-----------------+-----------+------------------+----------+---------------+--------+---------------+------+--------------------+--------------+---------+------+------------------------+---+-------------------+----------+------------------+-----------------+--------------+------------------------+-----------+---------------+---------+----------------+--------------+------------+--------------------+-------------------+------------------+-------------------------+--------------+------------------+-------------------+-----------------+----------------+---------------------+--------------------------+-----------------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALE